## Transforms can be applied to:
  - PIL Images
  - Tensors
  - ndarrays
  - Custom Data

Complete list of built-in transforms: [click here](https://pytorch.org/docs/stable/torchvision/transforms.html)

On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale

On Tensors
----------
LinearTransformation, Normalize, RandomErasing

Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage

Generic
-------
Use Lambda

Custom
------
Write own class

Compose multiple Transforms (list of transforms)
------------------------------------------------
composed = transforms.Compose([ReScale(256),
                               RandomCrop(224)])

same as: `torchvision.transforms.Compose(ReScale(256),
                               RandomCrop(224)])`

or, `torchvision.transforms.ReScale(256)`and,
    `torchvision.transforms.RandomCrop(224)`


In [1]:
"""import necessary modules"""
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [2]:
""""define a class to initialize or download a dataset"""
class WineDataset(Dataset):

    def __init__(self, transform=None):
        xy = np.loadtxt('wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # note that we do not convert to tensor here
        self.x_data = xy[:, 1:]
        self.y_data = xy[:, [0]]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.n_samples

In [3]:
"""
Custom Transforms
implement __call__(self, sample)
"""
class ToTensor:
    # Convert ndarrays to Tensors
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

class MulTransform:
    # multiply inputs with a given factor
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

In [14]:
print('Without Transform:')
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, labels = first_data
print("Features type: {}\nLabels type: {}".format(type(features), type(labels)))
print("\nFeatures:\n{}\nLabels:\n{}".format(features, labels))

Without Transform:
Features type: <class 'numpy.ndarray'>
Labels type: <class 'numpy.ndarray'>

Features:
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
Labels:
[1.]


In [16]:
print('With Tensor Transform:')
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print("Features type: {}\nLabels type: {}".format(type(features), type(labels)))
print("\nFeatures:\n{}\nLabels:\n{}".format(features, labels))

With Tensor Transform:
Features type: <class 'torch.Tensor'>
Labels type: <class 'torch.Tensor'>

Features:
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
Labels:
tensor([1.])


In [17]:
print('With Tensor and Multiplication Transform:')
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed) # compile a list of transforms
first_data = dataset[0]
features, labels = first_data
print("Features type: {}\nLabels type: {}".format(type(features), type(labels)))
print("\nFeatures:\n{}\nLabels:\n{}".format(features, labels))

With Tensor and Multiplication Transform:
Features type: <class 'torch.Tensor'>
Labels type: <class 'torch.Tensor'>

Features:
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03])
Labels:
tensor([1.])
